In this notebook, we compute the remaining symmetry groups by drilling out the systole.

In [1]:
import snappy

def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0),-1)
        for i in range(tries):
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
                break
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations

def better_symmetry_group(M,index=100,num_pos_triang=1):
    '''
    This function computes the symmetry group of the input manifold. 
    If the second entry is True it is proven to be the symmetry group.
    '''
    full=False
    S='unclear'
    try:
        S=M.symmetry_group()
        full=S.is_full_group()
        if full:
            return (S,full)
    except (ValueError, RuntimeError, snappy.SnapPeaFatalError):
        pass
    pos_triang=find_positive_triangulations(M,tries=index,number=num_pos_triang)
    if pos_triang==[]:
        randomizeCount=0
        while randomizeCount<index and full==False:
            try:
                S=M.symmetry_group()
                full=S.is_full_group()
                if full:
                    return (S,full)
                M.randomize()
                randomizeCount=randomizeCount+1
            except (ValueError, RuntimeError, snappy.SnapPeaFatalError):
                M.randomize()
                randomizeCount=randomizeCount+1
    for X in pos_triang:
        randomizeCount=0
        while randomizeCount<index and full==False:
            try:
                S=X.symmetry_group()
                full=S.is_full_group()
                if full:
                    return (S,full)
                X.randomize()
                randomizeCount=randomizeCount+1
            except (ValueError, RuntimeError, snappy.SnapPeaFatalError):
                X.randomize()
                randomizeCount=randomizeCount+1
    return (S,full)

def better_systole_word(M,index=100,length=1):
    '''
    Computes the word of the systole of a manifold. 
    It returns False if SnapPy cannot compute the systole.
    '''
    randomizeCount=0
    while randomizeCount<index:
        try:
            return M.length_spectrum(cutoff=length,include_words=True)[0].word
        except:
            M.randomize()
            randomizeCount=randomizeCount+1
    return False

def symmetry_group_via_drill(mfd,high_precision=False,maxlength=1,tries=10):
    '''
    Computes the length spectrum of a given input manifold M up to a given maximal length, 
    drills out the shortest geodesic, and computes the symmetry group of that drilled manifold.
    Since every symmetry of M sends the shortest geodesic to itself, this is the symmetry group of M. 
    '''
    M=snappy.Manifold(mfd)
    if high_precision:
        M=M.high_precision()
    for i in range(tries):
        try:
            systole=better_systole_word(M,length=maxlength)
            P=M.drill_word(systole)
            return better_symmetry_group(P)
        except:
            M.randomize()
    return False

In [2]:
import csv
unclear=[]
with open('unclear_knots.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        unclear.append(row)
unclear=unclear[1:]
len(unclear)

17825

In [3]:
import time
start_time=time.time()
still_unclear=[]
zero_surgery_is_isotopic_to_mirror=[]

for knot in unclear:
    if unclear.index(knot)%250==0:
        print('Number of knots handled:',unclear.index(knot))
    K=snappy.Manifold(knot[0])
    K.dehn_fill((0,1))
    S=symmetry_group_via_drill(K)
    if S==False:
        still_unclear.append(knot[0])
    else:
        if S[1]==False:
            still_unclear.append(knot[0])
        else:
            if S[0].is_amphicheiral():
                zero_surgery_is_isotopic_to_mirror.append(knot[0])

print('Time taken:',time.time()-start_time,' seconds')
print('Still unclear knots:',still_unclear)
print('Knot with zero surgery isotopic to mirror',zero_surgery_is_isotopic_to_mirror)

Number of knots handled: 0
Number of knots handled: 250
Number of knots handled: 500
Number of knots handled: 750
Number of knots handled: 1000
Number of knots handled: 1250
Number of knots handled: 1500
Number of knots handled: 1750
Number of knots handled: 2000
Number of knots handled: 2250
Number of knots handled: 2500
Number of knots handled: 2750
Number of knots handled: 3000
Number of knots handled: 3250
Number of knots handled: 3500
Number of knots handled: 3750
Number of knots handled: 4000
Number of knots handled: 4250
Number of knots handled: 4500
Number of knots handled: 4750
Number of knots handled: 5000
Number of knots handled: 5250
Number of knots handled: 5500
Number of knots handled: 5750
Number of knots handled: 6000
Number of knots handled: 6250
Number of knots handled: 6500
Number of knots handled: 6750
Number of knots handled: 7000
Number of knots handled: 7250
Number of knots handled: 7500
Number of knots handled: 7750
Number of knots handled: 8000
Number of knots 

In [4]:
K=snappy.Manifold('K15a5590')
K.dehn_fill((0,1))
S=symmetry_group_via_drill(K,high_precision=True)
S

(0, True)

We finish by demonstrating that 0 is a symmetry-exceptional slope for K14n3411 and K15n64176.

In [21]:
K=snappy.Manifold('K14n3411')
S=K.symmetry_group()
S

0

In [22]:
K.dehn_fill((0,1))
S=K.symmetry_group()
S

Z/2

In [23]:
K=snappy.Manifold('K15n64176')
S=K.symmetry_group()
S

0

In [24]:
K.dehn_fill((0,1))
S=K.symmetry_group()
S

Z/2